# Embedding widgets in popups

In [110]:
import os

from ipywidgets import IntSlider
from ipyleaflet import Map, Marker, Polyline

from ipyrest import Api
from ipyrest.responseviews import ResponseView, zoom_for_bbox

In [ ]:
class HereIsolinesView(ResponseView):
    """
    A view for the isolines from the HERE Routing API, see
    https://developer.here.com/documentation/routing/topics/request-isoline.html.
    """
    name = 'HereIsolines'
    mimetype_pats = ['application/json']

    def __init__(self, owner=None) -> None:
        super().__init__(owner=owner)
        self.slider = IntSlider(value=200, min=100, max=1000, step=100, description='Isoline (min)')
        self.slider.observe(self.obs_slider, names='value')
        self.range = 300

    def obs_slider(self, change) -> None:
        self.range = change.owner.value

    def move_marker(self, event, location) -> None:
        lat, lon = location['latitude'], location['longitude']

    def do_render(self, lat, lon, range):
        m = Map(center=(lat, lon))
        marker = Marker(location=(lat, lon), popup=self.slider)
        marker.on_move = self.move_marker
        m += marker
        mins, maxs = [], []
        for isoline in obj['response']['isoline']:
            shape = isoline['component'][0]['shape']
            path = [tuple(map(float, pos.split(','))) for pos in shape]
            m += Polyline(locations=path, color='red', weight=2, fill=True)
            mins.append(min(path))
            maxs.append(max(path))
        m.zoom = zoom_for_bbox(*min(mins), *max(maxs))
        self.data = m
        return m

    def render(self, resp):
        obj = resp.json()
        center = obj['response']['center']
        lat, lon = center['latitude'], center['longitude']
        m = Map(center=(lat, lon))
        marker = Marker(location=(lat, lon), popup=self.slider)
        marker.on_move = self.move_marker
        m += marker
        mins, maxs = [], []
        for isoline in obj['response']['isoline']:
            shape = isoline['component'][0]['shape']
            path = [tuple(map(float, pos.split(','))) for pos in shape]
            m += Polyline(locations=path, color='red', weight=2, fill=True)
            mins.append(min(path))
            maxs.append(max(path))
        m.zoom = zoom_for_bbox(*min(mins), *max(maxs))
        self.data = m
        return m

url = 'https://isoline.route.api.here.com' \
      '/routing/7.2/calculateisoline.json'
lat, lon = 52.5, 13.4
params = dict(
    app_id=os.getenv('HEREMAPS_APP_ID', 'rextLVOZWwI3G7bulbb3'), 
    app_code=os.getenv('HEREMAPS_APP_CODE', 'CexmP6l-DmHVYw0e8VXnhQ'),
    start=f'geo!{lat},{lon}',
    mode='fastest;car;traffic:disabled',
    rangetype='time', # time/distance
    range='300',  # seconds/meters
    resolution='20',  # meters
    #departure='now', # 2018-07-04T17:00:00+02
)
Api(url, params=params, additional_views=[HereIsolinesView])

## Example without Ipyrest

In [1]:
import os
from functools import partial

import requests
from ipywidgets import IntSlider, VBox, HBox, Dropdown, Button, HTML, Layout
from ipyleaflet import Map, Marker, Polyline

In [2]:
class DynamicIsolines(object):
    """
    A class to draw isolines dynamically around a marker on a map.

    Markers have a slider popup indicating the isoline range, and when
    changing the slider position or moving the marker the isoline is
    recalculated and updated while the one for the previous location
    and range is removed from the map.

    Isolines are cached for (lat, lon, range) keys and shown with red
    outlines when newly fetched via the API, or green ones when reused
    from previously cached API calls.
    
    Done:

    - add delete button
    - add slider for range
    - add slider for resolution

    TODO:

    - add toggle buttons for car/pedestrian mode
    - start with empty map
    - add isolines to map
    
    NOTES:
    
    - sliders don't have a name attribute
    - callbacks registered with marker.on_move() don't get a marker/owner argument 
    """
    def __init__(self, center=None, isolines=[]):
        self.isolines = isolines # {loc,range,obj}

        center = center or isolines[0]['loc'] 
        self.m = Map(center=center, zoom=12)
        self.m.on_interaction(self.map_click)

        self.cache = {} # (lat, lon, range) -> obj # TODO: add rangetype, resolution
        self.resize_map = False

        for iso in isolines:
            iso['from_cache'] = False
            
            range_slider = IntSlider(
                value=iso['range'],
                min=60, max=3600, step=60,
                description='Time (s)')
            obs_range_slider = partial(self.obs_range_slider, iso=iso)
            range_slider.observe(obs_range_slider, names='value')
            iso['range_slider'] = range_slider
            
            reso_slider = IntSlider(
                value=iso['resolution'],
                min=10, max=200, step=10,
                description='Resol. (m)')
            obs_reso_slider = partial(self.obs_reso_slider, iso=iso)
            reso_slider.observe(obs_reso_slider, names='value')
            iso['reso_slider'] = reso_slider
            
            obj = iso.get('obj', self.get_isoline_cached(iso))
            iso['obj'] = obj
            self.render_on_map(iso)

    # callbacks
    
    def rangetype_changed(self, change, iso):
        pass
        
    def obs_range_slider(self, change, iso):
        "Callback for events when a range slider was moved."
        self.clean(iso, incl_slider=False)
        iso['range'] = change.owner.value
        iso['obj'] = self.get_isoline_cached(iso)
        self.render_on_map(iso)

    def obs_reso_slider(self, change, iso):
        "Callback for events when a resolution slider was moved."
        self.clean(iso, incl_slider=False)
        iso['resolution'] = change.owner.value
        iso['obj'] = self.get_isoline_cached(iso)
        self.render_on_map(iso)

    def move_marker(self, event, location, iso) -> None:
        "Callback for events when marker was moved."
        self.clean(iso)
        iso['loc'] = tuple(location)
        iso['obj'] = self.get_isoline_cached(iso)
        self.render_on_map(iso)
    
    def click_delete(self, event, iso) -> None:
        "Callback for events when button was clicked."
        self.clean(iso)

    def map_click(self, **kwargs):
        "Callback for events when map was clicked."
        if kwargs.get('type') == 'click':
            iso = dict(
                loc=tuple(kwargs.get('coordinates')),
                range=300,
                resolution=100,
                from_cache=False
            )
            
            range_slider = IntSlider(
                value=iso['range'],
                min=60, max=3600, step=60,
                description='Time (s)')
            obs_range_slider = partial(self.obs_range_slider, iso=iso)
            range_slider.observe(obs_range_slider, names='value')
            iso['range_slider'] = range_slider
            
            reso_slider = IntSlider(
                value=iso['resolution'],
                min=10, max=200, step=10,
                description='Resol. (m)')
            obs_reso_slider = partial(self.obs_reso_slider, iso=iso)
            reso_slider.observe(obs_reso_slider, names='value')
            iso['reso_slider'] = reso_slider
            
            obj = iso.get('obj', self.get_isoline_cached(iso))
            iso['obj'] = obj

            self.m += self.make_marker(iso)

            self.isolines.append(iso)
            self.render_on_map(iso)
            
    # normal methods
    
    def clean(self, iso, incl_slider=True):
        "Remove current marker and/or isoline from map."
        if incl_slider:
            self.m -= iso['marker']
        self.m -= iso['polyline']
    
    def get_isoline_cached(self, iso):
        "Get isoline data from cache or by executing an API call if needed."
        if (iso['loc'], iso['range'], iso['resolution']) not in self.cache:
            obj = self.get_isoline(iso)
            self.cache[(iso['loc'], iso['range'], iso['resolution'])] = obj
            iso['from_cache'] = False
        else:
            obj = self.cache[(iso['loc'], iso['range'], iso['resolution'])]
            iso['from_cache'] = True
        iso['obj'] = obj
        return obj

    def get_isoline(self, iso):
        "Execute API call to get path data for isoline object."
        url = 'https://isoline.route.api.here.com' \
              '/routing/7.2/calculateisoline.json'
        params = dict(
            app_id=os.getenv('HEREMAPS_APP_ID', 'rextLVOZWwI3G7bulbb3'), 
            app_code=os.getenv('HEREMAPS_APP_CODE', 'CexmP6l-DmHVYw0e8VXnhQ'),
            start='geo!{lat},{lon}'.format(lat=iso['loc'][0], lon=iso['loc'][1]),
            mode='fastest;car;traffic:disabled',
            rangetype='time', # time/distance
            range=str(iso['range']),  # seconds/meters
            resolution=str(iso['resolution']),  # meters
            #departure='now', # 2018-07-04T17:00:00+02
        )
        return requests.get(url, params=params).json()

    def make_marker(self, iso):
        "Return a merker with popup to be put on a map."
        del_btn = Button(icon='trash', layout=Layout(width='30px'))
        click_delete = partial(self.click_delete, iso=iso)
        del_btn.on_click(click_delete)
        toggle = Dropdown(
            options=['Time', 'Distance'],
            value='Time',
            description='Type:',
            disabled=False,
            layout=Layout(width='180px')
        )
        popup = VBox([
            HBox([HTML('<b>Isoline</b>'), toggle, del_btn]), 
            iso['range_slider'], 
            iso['reso_slider']])
        popup = Map(center=(53, 14), zoom=12)
        lat, lon = iso['loc']
        marker = Marker(location=(lat, lon), popup=popup, name='isoline_marker')
        iso['marker'] = marker
        move_marker = partial(self.move_marker, iso=iso)
        marker.on_move(move_marker)
        return marker
        
    def render_on_map(self, iso):
        "Render isoline object on map."
        obj = iso['obj']
        center = obj['response']['center']
        lat, lon = center['latitude'], center['longitude']
        self.m += self.make_marker(iso)
        if self.resize_map:
            mins, maxs = [], []
        for isoline in obj['response']['isoline']:
            shape = isoline['component'][0]['shape']
            path = [tuple(map(float, pos.split(','))) for pos in shape]
            color = 'green' if iso['from_cache'] else 'red'
            polyline = Polyline(locations=path, color=color, weight=2, fill=True, name='isoline')
            iso['polyline'] = polyline
            self.m += polyline
            if self.resize_map:
                mins.append(min(path))
                maxs.append(max(path))
        if self.resize_map:
            self.m.zoom = zoom_for_bbox(*min(mins), *max(maxs))

In [3]:
# from sidecar import Sidecar
iso0 = dict(loc=(52.5, 13.4), range=300, resolution=20)
iso1 = dict(loc=(52.51, 13.41), range=600, resolution=100)
dyniso = DynamicIsolines(isolines=[iso0, iso1])
# with Sidecar(title='Foo'):
#     display(dyniso.m)
dyniso.m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [27]:
dyniso.clean()

## Example from Sylvain

From https://github.com/jupyter-widgets/ipyleaflet/issues/256

In [ ]:
from ipywidgets import HTML, IntSlider
from ipyleaflet import Map, Marker, Popup

center = (52.5, 13.4)
m = Map(center=center, zoom=9, close_popup_on_click=False)

marker = Marker(location=(52.501, 13.401), title='Foo', popup=IntSlider(description='Isoline (min)'))
m.add_layer(marker)

message = HTML(value='Try clicking the marker!')
popup = Popup(
    location=center,
    child=message,
    close_button=False,
    auto_close=False,
    close_on_escape_key=False
)

# m.add_layer(popup)
m

In [ ]:
def print_loc(event, location):
    entry = '{} {} \n'.format(str(event), str(location))
    open('widgets_in_popups.log', 'a').write(entry)
    marker.title = str(location)

marker.on_move(print_loc)

In [ ]:
! more widgets_in_popups.log

In [ ]:
marker.title = 'Bar'

In [ ]:
m.layers

In [ ]:
s = IntSlider(value=200, min=100, max=1000, step=100)

In [ ]:
s.observe()

In [ ]:
m = Map(center=(0, 0), zoom=1)

## Testing partials

In [34]:
def power(base, exponent):
    return base ** exponent

In [35]:
from functools import partial

square = partial(power, exponent=2)
cube = partial(power, exponent=3)

assert square(2) == 4
assert cube(2) == 8